In [1]:
from handler_collision_with_wall import handler_wall_collision as handler_wall_collision_cy
from data_structures.vector import MyVector
import numpy as np

In [2]:
def handler_wall_collision(position, speed, radius, wall_directing_vector, wall):
    # p index of the part
    x1, y1, x2, y2 = wall # x1<=x2 etc.

    # angle
    theta = np.sign(wall_directing_vector.y)*np.arccos(wall_directing_vector.x)

    # A and B
    sTheta, cTheta = np.sin(theta), np.cos(theta)
    B = -speed.x*sTheta+speed.y*cTheta
    if B == 0.0 : 
        return np.nan, None # TODO : should we add a tolerance ? It will never be equals to zero exactly...
    A = -position.x*sTheta+position.y*cTheta

    # new position of the wall in the new base
    y1_new_base = -x1*sTheta+y1*cTheta
    A_prime = A-y1_new_base
    # possible collision time :
    t_coll_1 = (-A_prime-radius)/B
    t_coll_2 = (-A_prime+radius)/B

    t_intersect = max(t_coll_1, t_coll_2)

    if(t_intersect > 0):
        # t_intersect = max(t_coll_1, t_coll_2) # because we are not anticipating them anymore
        # t_intersect = -A_prime/B # the time at which the disk crosses the line if its radius were radius=0.
        pos_intersect = position + t_intersect * speed

        wall_extrimity1_coordinates = MyVector(x1,y1)
        wall_extrimity2_coordinates = MyVector(x2,y2)
        # the reason why were are not using pos_intersect.norm is that it should be a 3D vector.
        dP1, dP2, dP3 = wall_extrimity2_coordinates-wall_extrimity1_coordinates, \
            pos_intersect-wall_extrimity1_coordinates, wall_extrimity2_coordinates-pos_intersect 
        norm_1 = dP1.norm()
        norm_2 = dP2.norm()
        # norm_3 = dP3.norm()
        qty=dP1.inner(dP2)/(norm_1*norm_1) # norm_1 cant be 0 because wall segments are not on same points.
        if(qty < 1 and qty > 0):
            return t_intersect, pos_intersect
    return np.nan, None

In [23]:
def wrapper(position, speed, radius, wall_directing_vector, wall):
    return handler_wall_collision_cy(position.x, position.y, speed.x, speed.y, radius, wall[0], wall[1], wall[2], wall[3], wall_directing_vector.x, wall_directing_vector.y)

In [24]:
%timeit handler_wall_collision(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0,1), [1,-1, 1, 1]) 

30.8 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
%timeit wrapper(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0,1), [1.,-1., 1., 1.]) 

1.89 µs ± 45.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
%timeit handler_wall_collision_cy(0.,0., 1., 0., 0.1, 1., -1., 1., 1., 1., 0.)


130 ns ± 4.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


~250 improvements which does not show when there are too much particles I think. 

In [36]:
print(handler_wall_collision(MyVector(1,1), MyVector(1,0), 0.1, MyVector(0,1), [1,-1, 1, 1]))
print(handler_wall_collision_cy(1.,1., 1., 0., 0.1, 1., -1., 1., 1., 0., 1.))
print(wrapper(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0.,1.), [1.,-1., 1., 1.]))

(nan, None)
(1000000000000000.0, None, None)
(1.1, 1.1, 0.0)


In [33]:
from time import time
N = 100000
t1 = time()
for k in range(N):
    wrapper(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0.,1.), [1.,-1., 1., 1.])
t2 = time()
t_wrapper = 

Time : 0.18272995948791504


In [34]:
N = 100000
t1 = time()
for k in range(N):
    handler_wall_collision(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0,1), [1,-1, 1, 1])
t2 = time()
print(f"Time : {t2-t1}")

Time : 3.056593418121338


In [35]:
from time import time
N = 100000
t1 = time()
for k in range(N):
    handler_wall_collision_cy(0.,0., 1., 0., 0.1, 1., -1., 1., 1., 0., 1.)
t2 = time()
print(f"Time : {t2-t1}")

Time : 0.01764225959777832
